In [28]:
from utils import read_lines
from collections import defaultdict
from itertools import combinations

def parse_program(s):
    return [int(x) for x in s.split(',')]

def parse_code(num):
    if num == 99:
        return num, [0]
    else:
        digits = []
        while num > 0:
            digits.append(num % 10)
            num //=10
        digits += [0] * (5-len(digits))
        opcode = digits[0]
        param_modes = digits[2:]
        return opcode, param_modes

def run(program, inputs):
    i = 0
    outputs = []
    extend_memory = defaultdict(int)
    relative_base = 0
    input_pos = 0
    commands_generator = generate_combinations()
    def read_memory(addr):
        if addr < len(program):
            return program[addr]
        else:
            return extend_memory[addr]
        
    def write_memory(idx, value, mode):
        addr = program[idx]
        if mode == 2:
            addr = relative_base + addr
        if addr < len(program):
            program[addr] = value
        else:
            extend_memory[addr] = value
    
    def get_oprand(idx, mode):
        if mode == 1:
            return program[idx]
        if mode == 2:
            addr = relative_base + program[idx]
        else:
            addr = program[idx]
        return read_memory(addr)

    while i < len(program):
        opcode, param_modes = parse_code(program[i])
        if opcode in (1, 2, 5, 6, 7, 8):
            oprand1 = get_oprand(i+1, param_modes[0])
            oprand2 = get_oprand(i+2, param_modes[1])
    
            if opcode == 1:
                write_memory(i+3, oprand1 + oprand2, param_modes[2])
                i+=4
            elif opcode == 2:
                write_memory(i+3, oprand1 * oprand2, param_modes[2])
                i+=4
            elif opcode == 5:
                if oprand1 != 0:
                    i = oprand2
                else:
                    i += 3
            elif opcode == 6:
                if oprand1 == 0:
                    i = oprand2
                else:
                    i += 3
            elif opcode == 7:
                write_memory(i+3, 1 if oprand1 < oprand2 else 0, param_modes[2])
                i+=4
            elif opcode == 8:
                write_memory(i+3, 1 if oprand1 == oprand2 else 0, param_modes[2])
                i+=4
        elif opcode ==3 :
            if not inputs:
                print_outputs(outputs)
                outputs = []
                commands = next(commands_generator)
                print(f'>>{commands}')
                
                inputs = encode_commands(commands)
                input_pos = 0

            v = inputs[input_pos]
            input_pos += 1
            if input_pos == len(inputs):
                inputs = []
            write_memory(i+1, v, param_modes[0])
            i += 2

        elif opcode == 4:
            oprand1 = get_oprand(i+1, param_modes[0])
            i += 2
            outputs.append(oprand1)
        elif opcode == 9:
            oprand1 = get_oprand(i+1, param_modes[0])
            relative_base += oprand1
            i+=2
        elif program[i] == 99:
            return outputs
        else:
            raise ValueError(f'illegal code {i} {program[i]}')

def print_outputs(outputs):
    print(''.join(chr(c) for c in outputs))

def generate_combinations():
    all_items = """jam
loom
mug
spool of cat6
prime number
food ration
fuel cell
manifold"""
    all_items = all_items.split('\n')
    cur_items = all_items[:]
    for r in range(len(all_items), 0, -1):
        for comb in combinations(all_items, r):
            commands = []
            for item in cur_items:
                if item not in comb:
                    commands.append(f'drop {item}')
            for item in comb:
                if item not in cur_items:
                    commands.append(f'take {item}')
            cur_items = list(comb)
            commands.append('north')
            yield commands


def part1(input_file):
    s = read_lines(input_file)[0]
    program = parse_program(s)
    initial_commands = [
        'east',
        'take food ration',
        'south',
        'take prime number',
        'north',
        'east',
        'take manifold',
        'east',
        'east',
        'take jam',
        'west',
        'north',
        'east',
        'take spool of cat6',
        'west',
        'north',
        'take fuel cell',
        'south',
        'south',
        'west',
        'west',
        'west',
        'north',
        'north',
        'west',
        'take mug',
        'east',
        'north',
        'east',
        'east',
        'take loom',
        'west',
        'west',
        'south',
        'south',
        'west',
        'north',
        'west',
        'inv'
    ]
    outputs = run(program, encode_commands(initial_commands))
    print_outputs(outputs)

def encode(func):
    ans = [ord(c) for c in func]
    ans.append(10)
    return ans

def encode_commands(cmds):
    return [c for cmd in cmds for c in encode(cmd)]


In [29]:
part1('inputs/day25.txt')




== Hull Breach ==
You got in through a hole in the floor here. To keep your ship from also freezing, the hole has been sealed.

Doors here lead:
- north
- east

Command?



== Hot Chocolate Fountain ==
Somehow, it's still working.

Doors here lead:
- east
- south
- west

Items here:
- food ration

Command?

You take the food ration.

Command?



== Science Lab ==
You see evidence here of prototype polymer design work.

Doors here lead:
- north

Items here:
- prime number

Command?

You take the prime number.

Command?



== Hot Chocolate Fountain ==
Somehow, it's still working.

Doors here lead:
- east
- south
- west

Command?



== Gift Wrapping Center ==
How else do you wrap presents on the go?

Doors here lead:
- east
- south
- west

Items here:
- manifold

Command?

You take the manifold.

Command?



== Navigation ==
Status: Stranded. Please supply measurements from fifty stars to recalibrate.

Doors here lead:
- north
- east
- west

Command?



== Kitchen ==
Everything's freez